#### Financial Analysis Project With Function Calling in Assistant API OpenAI

In [1]:
from openai import OpenAI
import json
from dotenv import find_dotenv , load_dotenv
import requests #to fetch
import os

_ : bool = load_dotenv(find_dotenv())

In [2]:
client : OpenAI = OpenAI()

##### We have used the dummy data through API 
https://site.financialmodelingprep.com/developer/docs?ref=mlq.ai#income-statements-financial-statements

In [3]:
key = os.environ["FMP_API_KEY"]


##### get Functions from above website

In [4]:
def Income_Statement(period):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/0000320193?period={period}&apikey={key}"
    res = requests.get(url)
    return json.dumps(res.json())

def Balance_sheet(period):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/0000320193?period={period}&apikey={key}" #period is annaual or quater
    res = requests.get(url)
    return json.dumps(res.json())

def cash_flow_statement(period):
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/0000320193?period={period}&apikey={key}"
    res = requests.get(url)
    return json.dumps(res.json())




In [11]:
import json

def show_json(message , obj):
    display(message , json.loads(obj.model_dump_json()))

##### Creating Assistant

In [24]:
from openai.types.beta import Assistant

assistant = client.beta.assistants.create(
    instructions="You are a financial analytics expert. Utilize the available functions to provide answers to questions",
    model="gpt-3.5-turbo-1106",
    tools=[{
        "type": "function",
        "function": {
            "name": "Income_Statement",
            "description": "Get Income Statement quater or annual  as per parameter or desire query",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {"type": "string", "description": "Income Statement Quarter or Annual"},
                },
                "required": ["period"]
            }
        }
    }, {
        "type": "function",
        "function": {
            "name": "Balance_sheet",
            "description": "Get Income Statement quater or annual  as per parameter or desire query",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {"type": "string", "description": "Balance Sheet of  Quarter or Annual"},
                },
                "required": ["period"]
            }
        }
    }, {
        "type": "function",
        "function": {
            "name": "cash_flow_statement",
            "description": "Get Income Statement quater or annual  as per parameter or desire query",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {"type": "string", "description": "Cash Flow Statement of  Quarter or Annual"},
                },
                "required": ["period"]
            }
        }
    }]
)


#### Create Thread

In [25]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_J9NUhl5VTQTAUpXam5EDd3dD', created_at=1710447891, metadata={}, object='thread')


#### Add Message to Thread

In [26]:
from openai.types.beta.threads.thread_message import ThreadMessage
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the quater income statment"
)

dict(message)

{'id': 'msg_j2tT2Jd7R33TbDzx4w7N6Mpz',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='Show me the quater income statment'), type='text')],
 'created_at': 1710447900,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_J9NUhl5VTQTAUpXam5EDd3dD'}

##### Create Run

In [27]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
    
)

dict(run)

{'id': 'run_SLKhhX2SD3IRkbE7GoxPXlgB',
 'assistant_id': 'asst_Je5akQNSNYSI8EO8yYbZGVWa',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1710447902,
 'expires_at': 1710448502,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a financial analytics expert. Utilize the available functions to provide answers to questions',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_J9NUhl5VTQTAUpXam5EDd3dD',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='Income_Statement', description='Get Income Statement quater or annual  as per parameter or desire query', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'description': 'Income Statement Quarter or Annual'}}, 'required': ['period']}), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='Balance_sheet', descriptio

#### Available Functions

In [28]:
available_functions = {
    "Income_Statement": Income_Statement,
    "Balance_sheet": Balance_sheet,
    "cash_flow_statement" : cash_flow_statement
} 

##### Polling for Updates and Calling Functions

In [29]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="Income_Statement","================================================================")
                  
                    if function_to_call.__name__ == "Income_Statement":
                        
                        response = function_to_call(
                        period=function_args.get("period"),
                        
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    elif function_to_call.__name__ == "Balance_sheet":
                        response = function_to_call(
                          period=function_args.get("period")
                          )
                        tool_outputs.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
                    elif function_to_call.__name__ == "cash_flow_statement":
                        response = function_to_call(
                          period=function_args.get("period")
                          )
                        tool_outputs.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break




requires_action .....
requires_action .....
Status:  requires_action


'submit_tool_outputs'

{'submit_tool_outputs': {'tool_calls': [{'id': 'call_JZq6wAI6mz0olxgMzOYqgnUy',
    'function': {'arguments': '{"period":"quarter"}',
     'name': 'Income_Statement'},
    'type': 'function'}]},
 'type': 'submit_tool_outputs'}

toolCalls present:
<function Income_Statement at 0x000001A254A54180> True ================================================================
[{'tool_call_id': 'call_JZq6wAI6mz0olxgMzOYqgnUy', 'output': '{"Error Message": "Special Endpoint : This endpoint is not available under your current subscription please visit our subscription page to upgrade your plan at https://site.financialmodelingprep.com/developer/docs/pricing"}'}] >>>>>
in_progress .....
Run is queued. Waiting...
completed .....
completed...........logic
Assistant: It seems that I don't have access to the quarterly income statement endpoint under the current subscription. You may need to upgrade the subscription to access this feature.

User: Show me the quater income statment



##### We can Also Use this

In [23]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    
                        response = function_to_call( **function_args)
        
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break




completed .....
completed...........logic
Assistant: It appears that I do not have access to the quarterly income statement data at the moment. If you have any other financial analysis or data queries, please feel free to ask and I'd be happy to assist you!

User: Show me the quater income statment

Assistant: It seems that I don't have access to the quarterly income statement data. If you'd like, I can provide other financial information or assist with a different query. Let me know how you'd like to proceed!

User: Show me the quater income statment

